# 1 Obtener datos de la API jsonplaceholder.typicode.com/todos.


In [1]:
import json
import requests

In [2]:
try:
    response = requests.get("https://jsonplaceholder.typicode.com/todos", timeout=3)
    # Si no lanza excepción por timeout o conexión, pasamos al siguiente if
    if response.status_code == 200:
        todos = json.loads(response.text)
        print("Datos recibidos correctamente.")
    else:
        print(f"Error en la respuesta: Status code {response.status_code}")
    
except requests.exceptions.RequestException as e:
    print(f"Ha ocurrido un error de conexión: {e}, pillamos la data del archivo descargado")
    with open ("saved_todos.json", "r") as file:
        todos = json.load(file)

Datos recibidos correctamente.


# 2 Crear un diccionario que mapea userId a la cantidad de tareas completadas.


In [5]:
todos_by_user = {}

for todo in todos:
    if todo["completed"]:
        todos_by_user[todo["userId"]] = todos_by_user.get(todo["userId"], 0) +1


Al iterar:
1. Primera vez que encuentras al usuario 2:  
   - `todos_by_user.get(2, 0)` → 0 (no existía)  
   - 0 + 1 = 1  
   - `todos_by_user[2] = 1`
2. Segunda vez:  
   - `todos_by_user.get(2, 0)` → 1  
   - 1 + 1 = 2  
   - `todos_by_user[2] = 2`
3. Tercera vez:  
   - `todos_by_user.get(2, 0)` → 2  
   - 2 + 1 = 3  
   - `todos_by_user[2] = 3`

Al final, `todos_by_user[2] = 3`, indicando que el usuario 2 completó 3 tareas. Eso **no es un error** sino la lógica natural de llevar un contador.



In [7]:
todos_by_user

{1: 11, 2: 8, 3: 7, 4: 6, 5: 12, 6: 6, 7: 9, 8: 11, 9: 8, 10: 12}

# 3 Encontrar qué usuarios completaron más tareas.

## Primero ordenamos el diccionario con SORTED()

In [8]:
top_users =sorted(todos_by_user.items(), key=lambda x:x[1], reverse = True)

In [9]:
top_users

[(5, 12),
 (10, 12),
 (1, 11),
 (8, 11),
 (7, 9),
 (2, 8),
 (9, 8),
 (3, 7),
 (4, 6),
 (6, 6)]

In [10]:
top_users_dict = dict(top_users)

In [11]:
top_users_dict

{5: 12, 10: 12, 1: 11, 8: 11, 7: 9, 2: 8, 9: 8, 3: 7, 4: 6, 6: 6}


Por qué usamos `sorted()` para encontrar al “top user”

Cuando tienes un diccionario así:
```python
todos_by_user = {
  1: 11,
  2: 8,
  3: 15,
  4: 7,
  ...
}
```
cada **clave** es un `userId` y cada **valor** es la cantidad de tareas completadas.

Para **encontrar** al usuario con más tareas completadas, queremos **ordenar** (sort) estos pares `(userId, cantidad)`. Sin embargo, un diccionario por sí mismo:
- **No** tiene un orden garantizado basado en valores.
- Sólo almacena la asociación `key -> value`.

Por eso hacemos:

```python
top_users = sorted(todos_by_user.items(), key=lambda x: x[1], reverse=True)
```
- `todos_by_user.items()` produce una lista de tuplas: `[(userId1, cantidad1), (userId2, cantidad2), ...]`.
- `key=lambda x: x[1]` indica que la **clave de ordenación** es el valor (cantidad) en la tupla `(userId, cantidad)`.
- `reverse=True` para que el orden sea **descendente** (más tareas completadas primero).

El resultado `top_users` es algo como:
```python
[
  (3, 15),
  (1, 11),
  (2, 8),
  (4, 7),
  ...
]
```
De esta forma, la tupla del índice `0` es `(3, 15)`, es decir, “el userId 3 con 15 tareas completadas”. ¡El top user!


## Después buscamos el primero 

In [12]:
max_complete = top_users[0][1]

In [13]:
max_complete

12

In [14]:
users = []

for user, num_complete in top_users:
    if num_complete < max_complete:
        break
    users.append(str(user))

In [15]:
max_users = " and ".join(users)

print(f"user(s) {max_users} completed {max_complete} TODOs")

user(s) 5 and 10 completed 12 TODOs


# 4 JSON file with all completed todo's of the TOP users

Veamos paso a paso en qué **consiste** la función `keep(todo)` y **por qué** se usa para filtrar.

```python
def keep(todo):
    is_completed = todo["completed"]
    has_max_count = todo["userId"] in users
    return is_completed and has_max_count
```

### 1. ¿Qué es `keep(todo)`?
- Es **una función de filtrado**. Dada una tarea (`todo`), devuelve `True` si la tarea debe “mantenerse” en la lista final, y `False` si no.
- `todo` es un **diccionario Python** con al menos estas claves: `userId`, `id`, `title`, `completed`.

### 2. ¿Por qué se llama dentro de `filter()`?
- Python tiene una función `filter(func, iterable)` que:
  1. Toma cada elemento (en este caso, cada `todo` de la lista `todos`),
  2. Llama a la función `func(todo)` (en tu ejemplo, `keep(todo)`),
  3. Si `keep(todo)` devuelve `True`, el elemento permanece en el “filtro”; si devuelve `False`, se descarta.
- Por tanto, `filter(keep, todos)` va iterando sobre cada “TODO” y se queda **solo** con aquellos para los que `keep(todo)` sea `True`.

### 3. ¿Qué hacen las variables `is_completed` y `has_max_count`?
```python
is_completed = todo["completed"]
has_max_count = todo["userId"] in users
```
1. `is_completed`: ¿La tarea está marcada como completada (`completed == True`)?  
   - `todo["completed"]` será `True` o `False`. 
2. `has_max_count`: ¿La tarea pertenece a un “usuario destacado” (por ejemplo, los que más tareas completaron)?  
   - `users` suele ser una lista de IDs, p. ej. `[5, 10]`.
   - Se verifica si `todo["userId"]` **está** en esa lista.  
   - Si `todo["userId"]` no forma parte de los top users, `has_max_count` será `False`.

### 4. El `return is_completed and has_max_count`

- En Python, `and` solo es `True` si **ambas** condiciones son `True`.  
- Significa: “Devuelve `True` si **la tarea está completada** y **el usuario es uno de los top**. En caso contrario, `False`”.  
- Esto garantiza que **solo** se filtren las tareas completadas de aquellos dos (o más) usuarios destacados.

### 5. El bug de tipos `int` vs. `str`
- En el video ocurre que los “top users” se guardaban como `["5", "10"]` (cadenas) mientras que `todo["userId"]` es un **entero** (`int`).
- Entonces `todo["userId"] in users` siempre daba `False` si estabas comparando `5 in ["5", "10"]`.
- La solución rápida es convertir uno de los lados para que coincidan en tipo, p. ej. `str(todo["userId"]) in users` o almacenar `users = [5, 10]` como enteros.

### 6. ¿Por qué se necesita una función separada?

Separar la lógica en una función `keep(todo)` tiene ventajas:

1. **Claridad**:  
   - El nombre “keep” indica que esta función decide si “mantener” o no el `todo` en la lista filtrada.
2. **Reutilización y mantenimiento**:  
   - Si luego quisieras filtrar de otra forma (por ejemplo, excluir títulos que tengan “velit”), podrías agregarlo dentro de `keep(todo)` sin romper el resto del código.
3. **Comodidad**:  
   - Al usar `filter(keep, todos)`, no necesitas escribir un bucle for manual para descartar los elementos. Python lo hace por ti.

### 7. Resultado final

- Una vez definida la función `keep(todo)`, se hace algo como:
  ```python
  filtered_todos = list(filter(keep, todos))
  ```
- Esa `filtered_todos` será una **lista** con **solo** aquellos `todo` que cumplan `completed == True` y `userId` en `[5, 10]`.
- Luego, con `json.dump(filtered_todos, data_file, indent=2)`, se guardan a `filtered_data_file.json`.

**En resumen**:
- `keep(todo)` es simplemente una **función** que decide **para cada tarea** si se queda (`True`) o se descarta (`False`).  
- Se basa en dos condiciones: que la tarea esté completada y que pertenezca a los usuarios que más completaron.  
- Usamos `filter(keep, todos)` para crear la lista filtrada.  
- El bug era un tema de **tipos** (int vs string), pero no un error en la lógica del filtrado o la serialización JSON.

¡Espero que ahora veas más clara la razón de ser y el funcionamiento de esta parte del código! Cualquier otra duda, aquí estoy para ayudarte.

In [22]:
todo

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

In [25]:
users

['5', '10']

In [33]:
#filtro de su el usuario
def keep(todo):
    is_completed = todo["completed"]
    has_max_count =  str(todo["userId"]) in users
    return is_completed and has_max_count

In [37]:
with open ("filtered_data_file.json", "w") as data_file:
    filtered_todos = list(filter(keep, todos))
    json.dump(filtered_todos, data_file, indent=2)